# IPL Chatbot

Original: Learn how to Build and Deploy a Chatbot in Minutes using Rasa (IPL Case Study!) - Mohd Sanad Zaki Rizvi, Analytics Vidhya https://www.analyticsvidhya.com/blog/2019/04/learn-build-chatbot-rasa-nlp-ipl)

**Objectives** To build a chatbot capable of fetching latest info about the ongoing IPL (Indian Premier League) matches from cricapi.com site.

<img src="images/ipl.jpg">

## Rasa Stack

- Lets you focus on improving the “Chatbot” part of your project 
- Default set up of Rasa works really well right out of the box for intent extraction and dialogue management
- LOCAL models, no API calls for intent extraction and dialogue management, except if your business logic needs external calls.

## Architecture of chatbot

<img src="images/rasa_full4.png">


- As soon as Rasa receives a message from the end user, it tries to predict or extract the “intent” and “entities” present in the message. This part is handled by Rasa NLU
- Once the user’s intent is identified, the Rasa Stack performs an action called action_match_news to get the updates from the latest IPL match
- Rasa then tries to predict what it should do next. This decision is taken considering multiple factors and is handled by Rasa Core
- In this example, Rasa is showing the result of the most recent match to the user. It has also predicted the next action that our model should take – to check with the user whether the chatbot was able to solve his/her query

## Installations
Official Documentation: https://rasa.com/docs/core/installation/
* Python
* Rasa Starter Pack
* Spacy Language Model


### Python
* Install Anaconda 4.2.0 for Python 3.5 or Ananconda 5.2.0 for Python 3.6 
* Windows Build tools: Make sure the Microsoft $VC++$ Compiler Visual Studio 2015 is installed, so python can compile any dependencies or https://visualstudio.microsoft.com/visual-cpp-build-tools/ Download the installer and select VC++ Build tools in the list.

(My Note: on main page of Anaconda is https://www.anaconda.com/distribution/ you see Python 3.7, which is problematic, gives HTTP error while creating env. I had to delete 3.7 and get 3.6. Btw, this site does not mention which Anaconda installer has which Python version. WHY? thats the most crucial info. Anyway, I have given them here.)

### Python Environment
* By installing conda, you get base or the root environment, which is the default.
* Practical tip: DO NOT install any packages in the root. ALWAYS create and env and install inside the new env.
* Env is needed esepcially for fragile packages like Python (its treated as a package) and rasa.
* So, **conda create -n rasa python=3.6** 

(My Note: Env management is again a sour point. It creates complete copy (deeeep) of python 3.6 and all other packages inside "envs" folder. Goes to 1.5 GB!! Can someone optimize it?)


### IPL bot code (for reference)

* Clone repo from https://github.com/mohdsanadzakirizvi/iplbot.git
* "Complete Version" gives latest running application (Some modifications are needed, mentioned below)

<!---
* Let’s take a look at the folder structure and the files that were created 
<img src="images/nlu.png">
* Install git, can git clone above package.
-->

* **WE WILL BE CODING IT FROM SCRATCH**
* Run following commands:

* **mkdir code; cd code;mkdir data**
* **activate rasa**
* **pip install -r iplbot_requirements.txt** or **conda env create -f environment.yml**


### Spacy Language Models
* Open cmd in Administrator mode and activate the env
* Download medium english by **python -m spacy download en_core_web_md** 
* Link with **python -m spacy link en_core_web_md en --force**


(My Note: First command will show that linking is done, but it may not have happened actually. So, do next step anyway.)

### Others
* Install nb_conda_kernels to let Jupyter Notebook see the new env
* Create account on cricapi.com (free, can login using Google account). Note down API Key and set it as CRICINFOAPI env variable

<!---
* Need Rasa Core SDK for custom actions **pip install rasa_core_sdk** and run it as **python -m rasa_core_sdk.endpoint --actions actions** where actions.py has all the custom actions
* Install pygraphviz by **conda install -c alubbock pygraphviz**
-->

### Importing Libraries

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

import rasa_nlu
import rasa_core
import spacy

## Rasa NLU: Extracting User Intent from a Message

The first thing we want to do is figure out the intent of the user. What does he or she want to accomplish? Let’s utilize Rasa and build an NLU model to identify user intent and its related entities.

### Preparing the NLU Training Data

Training data for extracting the user intent.
As you can see, the format of training data for ‘intent’ is quite simple in Rasa. You just have to:

- Start the line with “## intent:intent_name”
- Supply all the examples in the following lines

In [2]:
nlu_md = """
## intent:goodbye  
- Bye 
- Goodbye
- See you later
- Bye bot
- Goodbye friend
- bye
- bye for now
- catch you later
- gotta go
- See you
- goodnight
- have a nice day
- i'm off
- see you later alligator
- we'll speak soon
- end
- finish

## intent:greet
- Hi
- Hey
- Hi bot
- Hey bot
- Hello
- Good morning
- hi again
- hi folks
- hi Mister
- hi pal!
- hi there
- greetings
- hello everybody
- hello is anybody there
- hello robot
- who are you?
- what are you?
- what's up
- how do you do?

## intent:thanks
- Thanks
- Thank you
- Thank you so much
- Thanks bot
- Thanks for that
- cheers
- cheers bro
- ok thanks!
- perfect thank you
- thanks a bunch for everything
- thanks for the help
- thanks a lot
- amazing, thanks
- cool, thanks
- cool thank you

## intent:affirm
- y
- Y
- yes
- yes sure
- absolutely
- for sure
- yes yes yes
- definitely
- yes, it did.

## intent:current_matches
- what are the current matches
- can you list the matches in ipl 2019
- which cricket match is happening right now
- which ipl match is next
- which teams are playing next in ipl
- which team will play next in ipl
- tell me some ipl news
- i want ipl updates
- can you give me ipl latest updates
- what are the latest match updates
- who won the last ipl match
- which teams are competing in the next match
- how is ipl going
- what was the result of the last match
- when is the next match

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really
- n
- N
"""

%store nlu_md > data/nlu_data.md

Writing 'nlu_md' (str) to file 'data/nlu_data.md'.


You can include as many examples as you want for each intent. In fact, make sure to include slangs and short forms that you use while texting. The idea is to make the chatbot understand the way we type text. Feel free to refer to the complete version where I have given plenty of examples for each intent type.

### Defining the NLU Model Configuration

This file lets us create a text processing pipeline in Rasa. Luckily for us, Rasa comes with two default settings based on the amount of training data we have:
- “spacy_sklearn” pipeline if you have less than 1000 training examples
- “tensorflow_embedding” if you have a large amount of data

In [4]:
config = """
language: "en"
pipeline: spacy_sklearn
""" 

%store config > config/nlu_config.yml

Writing 'config' (str) to file 'config/nlu_config.yml'.


### Training the NLU Classifier Model

On command line you can run following command:

**python -m rasa_nlu.train -c nlu_config.yml --data data/nlu_data.md -o models --fixed_model_name nlu --project current --verbose**

Or programmatically you can write code

In [5]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("data/nlu_data.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config/nlu_config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






INFO:absl:Entry Point [tensor2tensor.envs.tic_tac_toe_env:TicTacToeEnv] registered with id [T2TEnv-TicTacToeEnv-v0]




INFO:rasa.nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa_nlu.model:Starting to train component SpacyNLP
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component SpacyTokenizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component SpacyFeaturizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Start

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished
C:\Users\YogeshKulkarni\AppData\Local\conda\conda\envs\rasa\lib\site-packages\sklearn\model_selection\_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Successfully saved model into 'D:\Yogesh\ToDos\Teaching\TeachingDataScience\Jupyter\models\nlu\current'


### Evaluating the NLU model on a random text (first way)

Let’s test how good our model is performing by giving it a sample text that it hasn’t been trained on for extracting intent. 

In [6]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("what is happening in the cricket world these days?"))

{
  "intent": {
    "name": "current_matches",
    "confidence": 0.5398919216672282
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "current_matches",
      "confidence": 0.5398919216672282
    },
    {
      "name": "greet",
      "confidence": 0.258883611593913
    },
    {
      "name": "goodbye",
      "confidence": 0.08764150338897336
    },
    {
      "name": "affirm",
      "confidence": 0.05091891706981237
    },
    {
      "name": "deny",
      "confidence": 0.032140384032668654
    },
    {
      "name": "thanks",
      "confidence": 0.030523662247404274
    }
  ],
  "text": "what is happening in the cricket world these days?"
}


Not only does our NLU model perform well on intent extraction, but it also ranks the other intents based on their confidence scores. This is a nifty little feature that can be really useful when the classifier is confused between multiple intents.

### Evaluating the NLU model on a random text (2nd way)
Let’s test how good our model is performing by giving it a sample text that it hasn’t been trained on for extracting intent. You can open an iPython/Python shell and follow the following steps:

In [8]:
from rasa_nlu.model import Interpreter
nlu_model = Interpreter.load('./models/nlu/current')
nlu_model.parse('what is happening in the cricket world these days?')

INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.


{'intent': {'name': 'current_matches', 'confidence': 0.5398919216672282},
 'entities': [],
 'intent_ranking': [{'name': 'current_matches',
   'confidence': 0.5398919216672282},
  {'name': 'greet', 'confidence': 0.258883611593913},
  {'name': 'goodbye', 'confidence': 0.08764150338897336},
  {'name': 'affirm', 'confidence': 0.05091891706981237},
  {'name': 'deny', 'confidence': 0.032140384032668654},
  {'name': 'thanks', 'confidence': 0.030523662247404274}],
 'text': 'what is happening in the cricket world these days?'}

### Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [10]:
from rasa_nlu.test import run_evaluation

run_evaluation("data/nlu_data.md", model_directory)

INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa_nlu.test:Running model for predictions:
100%|█████████████████████████████████████████████████████████████████████████████████| 83/83 [00:00<00:00, 258.79it/s]
INFO:rasa_nlu.test:Intent evaluation results:
INFO:rasa_nlu.test:Intent Evaluation: Only considering those 83 examples that have a defined intent out of 83 examples
INFO:rasa_nlu.test:F1-Score:  1.0
INFO:rasa_nlu.test:Precision: 1.0
INFO:rasa_nlu.test:Accuracy:  1.0
INFO:rasa_nlu.test:Classification report: 
                 precision    recall  f1-score   support

           deny       1.00      1.00      1.00         8
current_matches       1.00      1.00      1.00        15
         thanks       1.00      1.00      1.00        15
        goodbye       1.00      1.00      1.00        17
         affirm       1.00      1.00      1.00         9
          greet       1.00      1.00      1.00        19

      micro avg       1.00      1.00  

{'intent_evaluation': {'predictions': [{'text': 'Bye',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.9585333997253026},
   {'text': 'Goodbye',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.8738052021877047},
   {'text': 'See you later',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.784749061819731},
   {'text': 'Bye bot',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.7992521352304371},
   {'text': 'Goodbye friend',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.8169194572692575},
   {'text': 'bye',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.9585333997253026},
   {'text': 'bye for now',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.7526951795769953},
   {'text': 'catch you later',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.7557877977441877},
   {'text': 'gotta go',
    

## Rasa Core: Making Interactive Conversations

One of the most important aspects of a chatbot application is its ability to be interactive. 

### Designing the conversational flow

Think of the simplest conversation our chatbot can have with a user. What would be the flow of such a conversation?

---
Me: Hi

Iplbot: Hey! How may I help you?

Me: What was the result of the last match?

Iplbot: Here are some IPL quick info:
1.The match between Rajasthan Royals and Delhi Capitals was recently held and Delhi Capitals won.
2.The next match is Warriors vs Titans on 22 April 2019

Iplbot: Did that help you?

Me: yes, thank you!

Iplbot: Glad that I could help!

---
Let’s see how we can teach a simple conversation like that to Rasa:


The general format is:

This is called a user story path. I have provided a few stories in the data/stories.md file for your reference. This is the training data for Rasa Core.

### Writing  Stories

The way it works is:

- Give some examples of sample story paths that the user is expected to follow
- Rasa Core combines them randomly to create more complex user paths
- It then builds a probabilistic model out of that. This model is used to predict the next action Rasa should take

<img src="images/conversation_flow.png">

The above illustration might look complicated, but it’s simply listing out various possible user stories that I have taught Rasa. Here are a few things to note from the above graph:

- Except for the START and END boxes, all the colored boxes indicate user intent
- All the white boxes are actions that the chatbot performs
- Arrows indicate the flow of the conversation
- action_match_news is where we hit the CricAPI to get IPL information

In [11]:
stories_md = """
## news path 1
* greet
  - utter_greet
* current_matches
  - action_match_news
  - utter_did_that_help
* affirm or thanks
  - utter_gratitude
* goodbye
  - utter_goodbye

## news path 2
* current_matches
  - action_match_news
  - utter_did_that_help
* affirm or thanks
  - utter_gratitude
* goodbye
  - utter_goodbye

## news path 3
* greet
  - utter_greet
* current_matches
  - action_match_news
  - utter_did_that_help
* deny
  - utter_ask_again
* current_matches
  - action_match_news
  - utter_did_that_help
* affirm or thanks
  - utter_gratitude
* goodbye
  - utter_goodbye

## greet path
* greet
  - utter_greet

## goodbye path
* goodbye
  - utter_goodbye
"""

%store stories_md > data/stories.md

Writing 'stories_md' (str) to file 'data/stories.md'.


Now, generate a similar graph for your stories using the following command:

**python -m rasa_core.visualize -d domain.yml -s data/stories.md -o graph.html**

This is very helpful when debugging the conversational flow of the chatbot.

### Defining the Domain

The domain is the world of your chatbot. It contains everything the chatbot should know, including:

- All the actions it is capable of doing
- The intents it should understand
- The template of all the utterances it should tell the user, and much more

In [12]:
domain_yml = """
actions:
- utter_greet
- utter_did_that_help
- utter_goodbye
- action_match_news
- utter_default
- utter_gratitude
- utter_ask_again

intents:
- goodbye
- greet
- thanks
- current_matches
- affirm
- deny

templates:
  utter_greet:
  - text: "Hey! What can I do for you?"
  utter_did_that_help:
  - text: "Did that help you?"
  - text: "I hope that solved your query"
  utter_goodbye:
  - text: "Bye"
  utter_default:
  - text: "I am sorry, I didn't get that. Could you please repeat your query?"
  - text: "I am not sure what you are aiming for."
  utter_gratitude:
  - text: "Glad that I could be of help to you!\nBye"
  utter_ask_again:
  - text: "Okay! Let's start again, please tell me what do you need?"
  - text: "No issues! Let's try this again.\n Please repeat your query?"
"""

%store domain_yml > config/domain.yml

Writing 'domain_yml' (str) to file 'config/domain.yml'.


### Setting Policies

Rasa Core generates the training data for the conversational part using the stories we provide. It also lets you define a set of policies to use when deciding the next action of the chatbot. These policies are defined in the policies.yml file.

In [13]:
policies_yml = """
policies:
  - name: KerasPolicy
    epochs: 100
    max_history: 5
  - name: FallbackPolicy
    fallback_action_name: 'action_default_fallback'
  - name: MemoizationPolicy
    max_history: 5
"""

%store policies_yml > config/policies.yml

Writing 'policies_yml' (str) to file 'config/policies.yml'.


Notes:
- KerasPolicy uses a neural network implemented in Keras to select the next action. The default architecture is based on an LSTM (Long Short Term Memory) model
- MemoizationPolicy memorizes the conversations in your training data. It predicts the next action with confidence 1.0 if this exact conversation exists in the training data, otherwise, it predicts ‘None’ with confidence 0.0
- FallbackPolicy invokes a fallback action if the intent recognition has confidence below nlu_threshold or if none of the dialogue policies predict action with confidence higher than core_threshold
- One important hyperparameter for Rasa Core policies is the max_history. This controls how much dialogue history the model looks at to decide which action to take next

###  Custom Actions

Using CricAPI for fetching IPL related news. It is free for 100 requests per day, which (I hope) is more than enough to satiate that cricket crazy passion you have.

You need to first signup on the website to get access to their API:
https://www.cricapi.com/

You should be able to see your API Key once you are logged in:

<img src="images/lala-1140x399.png">

Modifications to original code:

- Instead of showing API key here it has been stored in ENV variable and fetched here
- Key "toss_winner_team" was subsituted into depreceated key

In [14]:
actions_py="""
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from datetime import datetime

import logging
import requests
import json
import os
from rasa_core_sdk import Action

logger = logging.getLogger(__name__)

API_URL = "https://cricapi.com/api/"
API_KEY = os.environ.get('CRICINFOAPI')

class ApiAction(Action):
    def name(self):
        return "action_match_news"

    def run(self, dispatcher, tracker, domain):
        print(API_URL + "matches" + "?apikey=" + API_KEY)
        res = requests.get(API_URL + "matches" + "?apikey=" + API_KEY) #, verify=False
        if res.status_code == 200:
            data = res.json()["matches"]
            recent_match = data[0]
            upcoming_match = data[1]
            upcoming_match["date"] = datetime.strptime(upcoming_match["date"], "%Y-%m-%dT%H:%M:%S.%fZ")
            next_date = upcoming_match["date"].strftime("%d %B %Y")

            out_message = "Here some IPL quick info: 1.The match between {} and {} was recently held and {} won the toss.".format(recent_match["team-1"], recent_match["team-2"], recent_match["toss_winner_team"])

            dispatcher.utter_message(out_message)

            out_message = "2.The next match is {} vs {} on {}".format(upcoming_match["team-1"], upcoming_match["team-2"], next_date)

            dispatcher.utter_message(out_message)

            return []
"""
%store actions_py > actions.py

Writing 'actions_py' (str) to file 'actions.py'.


Need endpoints yml to execute the actions server.

Note: If you have external API call, like REST, need to have "webhook" word at the end, else nothing.

My own query on this topic: https://forum.rasa.com/t/rasa-core-sdk-not-working/9228

In [15]:
endpoints_yml = """
#action_endpoint:
#  url: "http://localhost:5055/webhook"
  
action_endpoint:
  url: http://localhost:5055/webhook

#nlg:
#  url: http://localhost:5056/nlg

core_endpoint:
  url: http://localhost:5005
"""
%store endpoints_yml > endpoints.yml

Writing 'endpoints_yml' (str) to file 'endpoints.yml'.


In a separate shell (cmd for Windows):
- **activate rasa**
- come to directory where actions.py is and then run
- **python -m rasa_core_sdk.endpoint --actions actions**

This way, custom action server starts ...

In [16]:
# import sys
# python = sys.executable
# !{python} -m rasa_core_sdk.endpoint --actions actions

###  Visualising the Training Data

In [27]:

from IPython.display import Image, display
from rasa_core.agent import Agent
%matplotlib inline

agent = Agent('config/domain.yml')
agent.visualize("data/stories.md", "story_graph.png", max_history=2)
i = Image(filename="story_graph.png")
display(i)

FileNotFoundError: [Errno 2] No such file or directory: 'story_graph.png'

### Training a Dialogue Model

You can train the model using the following command:

**python -m rasa_core.train -d domain.yml -s data/stories.md -o models/current/dialogue -c policies.yml**

Or do it programmatically as:

In [45]:
import asyncio
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy, MappingPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

# agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

agent = Agent('config/domain.yml', policies = [MemoizationPolicy(max_history=2), KerasPolicy(validation_split=0.2,epochs=200),MappingPolicy()])
# loop = asyncio.get_event_loop() # https://github.com/RasaHQ/rasa/issues/3649
# training_data = loop.run_until_complete(agent.load_data('data/stories.md'))
training_data = await agent.load_data('data/stories.md') # https://forum.rasa.com/t/call-agent-train-occurs-typeerror-coroutine-object-is-not-iterable/10135/10
# loading our neatly defined training dialogues
# training_data = agent.load_data('data/stories.md')

agent.train(training_data)
# FOLLOWING WAY OF giving arguments is depreacted, instead, give these params as arguments to KerasPolicy
# agent.train(
#     training_data,
#     validation_split=0.0,
#     epochs=200
# )

agent.persist('models/current/dialogue')

Processed trackers: 100%|█████████████████████████████████████████████████| 5/5 [00:00<00:00, 135.49it/s, # actions=15]
Processed actions: 15it [00:00, 1157.01it/s, # examples=15]
Processed trackers: 100%|█████████████████████████████████████████████| 121/121 [00:01<00:00, 103.08it/s, # actions=64]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 21)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                6912      
_________________________________________________________________
dense (Dense)                (None, 15)                495       
_________________________________________________________________
activation (Activation)      (None, 15)                0         
Total params: 7,407
Trainable params: 7,407
Non-trainable params: 0
_________________________________________________________________


INFO:rasa.core.policies.keras_policy:Fitting model with 64 total samples and a validation split of 0.2


Epoch 1/200
64/64 [==============================] - ETA: 0s - loss: 2.7082 - acc: 0.156 - 0s 5ms/sample - loss: 2.6752 - acc: 0.1875
Epoch 2/200
64/64 [==============================] - ETA: 0s - loss: 2.6703 - acc: 0.250 - 0s 171us/sample - loss: 2.6521 - acc: 0.2344
Epoch 3/200
64/64 [==============================] - ETA: 0s - loss: 2.6162 - acc: 0.343 - 0s 187us/sample - loss: 2.6043 - acc: 0.2812
Epoch 4/200
64/64 [==============================] - ETA: 0s - loss: 2.6018 - acc: 0.343 - 0s 171us/sample - loss: 2.5812 - acc: 0.3125
Epoch 5/200
64/64 [==============================] - ETA: 0s - loss: 2.5892 - acc: 0.250 - 0s 187us/sample - loss: 2.5596 - acc: 0.2656
Epoch 6/200
64/64 [==============================] - ETA: 0s - loss: 2.5486 - acc: 0.406 - 0s 187us/sample - loss: 2.5270 - acc: 0.3594
Epoch 7/200
64/64 [==============================] - ETA: 0s - loss: 2.5618 - acc: 0.281 - 0s 156us/sample - loss: 2.5232 - acc: 0.2969
Epoch 8/200
64/64 [==============================]

Epoch 61/200
64/64 [==============================] - ETA: 0s - loss: 1.5667 - acc: 0.437 - 0s 125us/sample - loss: 1.4797 - acc: 0.4688
Epoch 62/200
64/64 [==============================] - ETA: 0s - loss: 1.5387 - acc: 0.468 - 0s 156us/sample - loss: 1.4803 - acc: 0.4844
Epoch 63/200
64/64 [==============================] - ETA: 0s - loss: 1.4826 - acc: 0.468 - 0s 171us/sample - loss: 1.4343 - acc: 0.4531
Epoch 64/200
64/64 [==============================] - ETA: 0s - loss: 1.4653 - acc: 0.468 - 0s 140us/sample - loss: 1.4203 - acc: 0.4844
Epoch 65/200
64/64 [==============================] - ETA: 0s - loss: 1.4581 - acc: 0.500 - 0s 140us/sample - loss: 1.4025 - acc: 0.5000
Epoch 66/200
64/64 [==============================] - ETA: 0s - loss: 1.4562 - acc: 0.500 - 0s 125us/sample - loss: 1.4104 - acc: 0.4688
Epoch 67/200
64/64 [==============================] - ETA: 0s - loss: 1.4536 - acc: 0.468 - 0s 140us/sample - loss: 1.3711 - acc: 0.5000
Epoch 68/200
64/64 [=====================

64/64 [==============================] - ETA: 0s - loss: 0.8451 - acc: 0.906 - 0s 140us/sample - loss: 0.8513 - acc: 0.8438
Epoch 121/200
64/64 [==============================] - ETA: 0s - loss: 0.8866 - acc: 0.843 - 0s 156us/sample - loss: 0.8454 - acc: 0.8125
Epoch 122/200
64/64 [==============================] - ETA: 0s - loss: 0.9060 - acc: 0.875 - 0s 140us/sample - loss: 0.8513 - acc: 0.8594
Epoch 123/200
64/64 [==============================] - ETA: 0s - loss: 0.8512 - acc: 0.875 - 0s 156us/sample - loss: 0.8216 - acc: 0.8438
Epoch 124/200
64/64 [==============================] - ETA: 0s - loss: 0.8286 - acc: 0.875 - 0s 140us/sample - loss: 0.8519 - acc: 0.8594
Epoch 125/200
64/64 [==============================] - ETA: 0s - loss: 0.8410 - acc: 0.843 - 0s 156us/sample - loss: 0.8163 - acc: 0.8750
Epoch 126/200
64/64 [==============================] - ETA: 0s - loss: 0.8914 - acc: 0.812 - 0s 140us/sample - loss: 0.8137 - acc: 0.8750
Epoch 127/200
64/64 [===========================

64/64 [==============================] - ETA: 0s - loss: 0.4817 - acc: 0.937 - 0s 140us/sample - loss: 0.4544 - acc: 0.9531
Epoch 180/200
64/64 [==============================] - ETA: 0s - loss: 0.4845 - acc: 0.937 - 0s 140us/sample - loss: 0.4386 - acc: 0.9531
Epoch 181/200
64/64 [==============================] - ETA: 0s - loss: 0.4890 - acc: 0.937 - 0s 156us/sample - loss: 0.4463 - acc: 0.9688
Epoch 182/200
64/64 [==============================] - ETA: 0s - loss: 0.4399 - acc: 0.937 - 0s 125us/sample - loss: 0.4347 - acc: 0.9375
Epoch 183/200
64/64 [==============================] - ETA: 0s - loss: 0.4955 - acc: 0.937 - 0s 125us/sample - loss: 0.4482 - acc: 0.9531
Epoch 184/200
64/64 [==============================] - ETA: 0s - loss: 0.4895 - acc: 0.937 - 0s 156us/sample - loss: 0.4371 - acc: 0.9688
Epoch 185/200
64/64 [==============================] - ETA: 0s - loss: 0.4356 - acc: 0.968 - 0s 156us/sample - loss: 0.4148 - acc: 0.9688
Epoch 186/200
64/64 [===========================

INFO:rasa.core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/current/dialogue\core exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to 'D:\Yogesh\ToDos\Teaching\TeachingDataScience\Jupyter\models\current\dialogue\core'


## Running: Talk to your Bot

So we have the chatbot ready. It’s time to chat

One way is to run following command in shell (windows cmd)
- activate the environment,
- come to directory where actions.py is and then run
- **python -m rasa_core.run -d models/current/dialogue -u models/current/nlu --endpoints endpoints.yml**

Or else do programmatically like below

Both approaches expect rasa core sdk server running in a separate window, else **python -m rasa_core_sdk.endpoint --actions actions**

In [ ]:
# #Starting the Bot

# from rasa_core.agent import Agent
# agent = Agent.load('models/current/dialogue', interpreter=model_directory)
import IPython
from IPython.display import clear_output
from rasa_core.agent import Agent
from rasa_core.interpreter import NaturalLanguageInterpreter
# from rasa_core.utils import EndpointConfig
from rasa.utils.endpoints import EndpointConfig

import time

def load_assistant():
    messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
    model_dir = 'models/nlu/current' # or model_directory defined earlier
    interpreter = NaturalLanguageInterpreter.create(model_dir)
    endpoint = EndpointConfig('http://localhost:5055/webhook')
    agent = Agent.load('models/current/dialogue', interpreter=interpreter, action_endpoint = endpoint)

    print("Your bot is ready to talk! Type your messages here or send 'stop'")
    while True:
        a = input()
        if a == 'stop':
            break
        responses = agent.handle_text(a)
        for response in responses:
            print(response["text"])

In [ ]:
load_assistant()        

## What Next?

- Try to use different pipelines in Rasa Core, explore more Policies, fine-tune those models, 
- Check out what other features CricAPI provides, etc.
- Other APIs
- Slot filling
- Different languages (Hindi bot?)

